In [8]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os

In [9]:
folder = 'src/data'
os.listdir(folder)

['aisles.csv',
 'departments.csv',
 'orders.csv',
 'order_products__prior.csv',
 'order_products__train.csv',
 'products.csv']

In [10]:
products = pd.read_csv(folder + '/' + 'products.csv'); products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [11]:
departments = pd.read_csv(folder + '/' + 'departments.csv')
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [12]:
df = pd.merge(products, departments, how="left", on="department_id")[["product_id","product_name", "department"]]
df.head()

,product_id,product_name,department
0,1,Chocolate Sandwich Cookies,snacks
1,2,All-Seasons Salt,pantry
2,3,Robust Golden Unsweetened Oolong Tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen
4,5,Green Chile Anytime Sauce,pantry


In [13]:
db_uri = "blablabla"

In [14]:
from sqlmodel import SQLModel, Field, create_engine, Session
class Product(SQLModel, table=True):
    product_id: int = Field(default=None, primary_key=True) 
    product_name: str 
    department: str
    

engine = create_engine(db_uri)
SQLModel.metadata.create_all(bind=engine)

In [15]:
def df_to_sqlmodel(df: pd.DataFrame):
    objs = [Product(**row) for row in df.to_dict('records')]
    for obj in objs:
        obj.product_name = obj.product_name.lower()
    return objs

In [16]:
df_to_sqlmodel(df)

[Product(product_id=1, product_name='chocolate sandwich cookies', department='snacks'),
 Product(product_id=2, product_name='all-seasons salt', department='pantry'),
 Product(product_id=3, product_name='robust golden unsweetened oolong tea', department='beverages'),
 Product(product_id=4, product_name='smart ones classic favorites mini rigatoni with vodka cream sauce', department='frozen'),
 Product(product_id=5, product_name='green chile anytime sauce', department='pantry'),
 Product(product_id=6, product_name='dry nose oil', department='personal care'),
 Product(product_id=7, product_name='pure coconut water with orange', department='beverages'),
 Product(product_id=8, product_name="cut russet potatoes steam n' mash", department='frozen'),
 Product(product_id=9, product_name='light strawberry blueberry yogurt', department='dairy eggs'),
 Product(product_id=10, product_name='sparkling orange juice & prickly pear beverage', department='beverages'),
 Product(product_id=11, product_name=

In [17]:
with Session(engine) as sess:
    for row in df_to_sqlmodel(df):
        sess.add(row)
    
    sess.commit()
